In [1]:
import sklearn
import pymc as pm
import arviz as az

In [2]:
import sklearn.datasets
import pandas as pd


irisX, cat = sklearn.datasets.load_iris(return_X_y = True,as_frame=True)

In [ ]:
irisX

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [9]:
# center the data
irisX = irisX - irisX.mean()

In [ ]:
pm.__version__

'5.21.0'

In [4]:
cat.unique()

array([0, 1, 2])

In [11]:
nclass = len(cat.unique())
nparam = 4


with pm.Model() as iris_model:
    X_data = pm.Data('X_data', irisX)
    y_obs_data = pm.Data('y_obs_data', cat)
    alfa = pm.Normal('alfa', mu=0, sigma=10, shape=nclass)
    beta = pm.Normal('beta', mu=0, sigma=10, shape=(nparam, nclass))
    mu = pm.math.dot(X_data, beta) + alfa
    yl = pm.Categorical('obs', logit_p=mu, observed=y_obs_data)
 

with iris_model:
    idata = pm.sample()

 


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alfa, beta]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 8 seconds.
There were 3 divergences after tuning. Increase `target_accept` or reparameterize.


In [12]:
az.summary(idata, hdi_prob=0.95, round_to=2)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alfa[0],-4.18,7.76,-19.30,11.41,0.15,0.13,2824.86,2635.60,1.0
alfa[1],10.48,6.55,-2.18,23.17,0.15,0.10,2037.17,2426.49,1.0
alfa[2],-6.52,6.84,-19.86,6.43,0.14,0.11,2330.51,2183.86,1.0
"beta[0, 0]",0.09,8.42,-16.49,16.08,0.16,0.15,2766.49,2536.85,1.0
"beta[0, 1]",1.78,6.63,-11.82,14.98,0.14,0.12,2356.34,2119.28,1.0
"beta[0, 2]",-1.29,6.71,-14.26,12.34,0.14,0.12,2318.47,2235.00,1.0
"beta[1, 0]",6.72,7.92,-9.24,21.79,0.15,0.12,2746.00,2660.66,1.0
"beta[1, 1]",-0.87,6.52,-12.82,12.36,0.14,0.11,2081.73,2069.76,1.0
"beta[1, 2]",-5.54,6.67,-17.95,7.89,0.14,0.12,2119.75,2297.79,1.0
"beta[2, 0]",-11.76,7.44,-26.17,3.26,0.15,0.12,2330.68,2114.60,1.0


Now try using pivot 